In [ ]:
import pandas as pd
import numpy as np
import keras as keras
trainFile_letras = "DatasetsIAO/Letras/sign_mnist_train.csv"
testFile_letras = "DatasetsIAO/Letras/sign_mnist_test.csv"
path_npy = "DatasetsIAO/Letras/"

# Estos tienen solo una sola salida. 24x24
letras_train = pd.read_csv(trainFile_letras)
letras_test = pd.read_csv(testFile_letras)

In [ ]:
print(letras_train.shape)
print(letras_test.shape)

In [ ]:
letras_train

In [ ]:
# Concatenamos las columnas para poder manejarlas mejor al hacer shuffle

from sklearn.utils import shuffle
# Aleatorizamos las imagenes
letras_train = shuffle(letras_train)

letras_val = letras_train[:7000]
letras_train = letras_train.drop(range(0, 7000), axis=0)

In [ ]:
letras_train_Y = letras_train['label']
letras_train_X = letras_train.drop('label', axis=1)

letras_test_Y = letras_test['label']
letras_test_X = letras_test.drop('label', axis=1)

letras_val_Y = letras_val['label']
letras_val_X = letras_val.drop('label', axis=1)

In [ ]:
letras_train_Y

In [ ]:
letras_train_X

In [ ]:
letras_val_X

In [ ]:
letras_val_Y

In [ ]:
letras_val

In [ ]:
from keras.utils import np_utils

# Hacemos el reshape para que las imagenes tengan el tamaño correcto
letras_train_X = letras_train_X.values.reshape(letras_train_X.shape[0], 28, 28, 1)
letras_test_X = letras_test_X.values.reshape(letras_test_X.shape[0], 28, 28, 1)
letras_val_X = letras_val_X.values.reshape(letras_val_X.shape[0], 28, 28, 1)


# Pasamos las salidas a one hot encoding. Aunque tenemos 24 clases
# obtenemos un array de 25 posiciones. Esto se debe a que la J (numero
# 9) también se tiene en cuenta, aunque no aparezca en ningún momento.
letras_train_Y = np_utils.to_categorical(letras_train_Y)
letras_test_Y = np_utils.to_categorical(letras_test_Y)
letras_val_Y = np_utils.to_categorical(letras_val_Y)

In [ ]:
print(letras_train_X.shape)
print(letras_train_Y.shape)

print(letras_test_X.shape)
print(letras_test_Y.shape)

print(letras_val_X.shape)
print(letras_val_Y.shape)

In [ ]:
from matplotlib import pyplot as plt

example = letras_train_X[1000].reshape(28, 28)

print(example.shape)
plt.imshow(example, cmap="gray")

In [ ]:
from matplotlib import pyplot as plt

example = letras_val_X[56].reshape(28, 28)

print(example.shape)
plt.imshow(example, cmap="gray")

In [ ]:
example = letras_train_X[1782].reshape(28, 28)

print(example.shape)
plt.imshow(example, cmap="gray")

In [ ]:
np.save(path_npy + "letras_train_X.npy", letras_train_X)
np.save(path_npy + "letras_train_Y.npy", letras_train_Y)
np.save(path_npy + "letras_test_X.npy", letras_test_X)
np.save(path_npy + "letras_test_Y.npy", letras_test_Y)
np.save(path_npy + "letras_val_X.npy", letras_val_X)
np.save(path_npy + "letras_val_Y.npy", letras_val_Y)